# Initial exploration of each data file
Pandas makes importing data from files easy. But sometimes the file contents are poorly formatted or can hold hidden surprises. Make sure that the data - and data types - are what you expect them to be before starting your analysis.

In [2]:
import pandas as pd
import numpy as np
import os
from os.path import join

cwd = os.getcwd()
data_path = join(cwd, '..', '..', 'data')

I sometimes find it helpful to change the Pandas viewing options for max rows and max columns

In [37]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

### Don't write absolute paths
An absolute path is something like `/Users/Home/Documents/GitHub/python-data-analysis-class/data/epa_emissions_2016.txt`. Or in Windows it might be `C:\Users\gschivley\Documents\GitHub\python-data-analysis-class\data\epa_emissions_2016.txt`.

Use relative paths and Python built-in tools to write paths.

In [3]:
print(cwd)
print(data_path)

/Users/Home/Documents/GitHub/python-data-analysis-class/notebooks/Pandas
/Users/Home/Documents/GitHub/python-data-analysis-class/notebooks/Pandas/../../data


## Set file paths

In [106]:
# Paths to each of the data files (epa emissions, eia capacity by generator, and eia generation)

epa_path = join(data_path, 'epa_emissions_2016.txt')
cap_path = join(data_path, '3_1_Generator_Y2016.xlsx')
gen_path = join(data_path, 'EIA923_Schedules_2_3_4_5_M_12_2016_Final_Revision.xlsx')

## Load EPA epa data
Lets load the file and see what needs to be done to make sure the data is in good shape and accessible.

In [107]:
epa = pd.read_csv(epa_path)

It looks like the header column is not well aligned with the data. States are showing up as the index and the first column is labeled **State**.

In [108]:
epa.head()

,State,Facility Name,Facility ID (ORISPL),Month,Year,Gross Load (MW-h),SO2 (tons),NOx (tons),CO2 (short tons),Heat Input (MMBtu)
AL,AMEA Sylacauga Plant,56018,1,2016,4534.0,0.014,2.229,3101.8,52585.8,NaN
AL,AMEA Sylacauga Plant,56018,2,2016,792.0,0.002,0.361,542.0,9186.0,NaN
AL,AMEA Sylacauga Plant,56018,3,2016,1498.0,0.005,0.677,1024.2,17365.5,NaN
AL,AMEA Sylacauga Plant,56018,4,2016,1405.0,0.005,0.586,884.1,14987.3,NaN
AL,AMEA Sylacauga Plant,56018,5,2016,1791.0,0.006,0.756,1145.1,19412.8,NaN


In [109]:
with open(epa_path) as f:
    f.read()

In [110]:
with open(epa_path) as f:
    head = [next(f) for x in range(5)]

for line in head:
    print(line) 

State, Facility Name, Facility ID (ORISPL), Month, Year, Gross Load (MW-h), SO2 (tons), NOx (tons), CO2 (short tons), Heat Input (MMBtu)

"AL","AMEA Sylacauga Plant","56018","1","2016","4534","0.014","2.229","3101.8","52585.8",

"AL","AMEA Sylacauga Plant","56018","2","2016","792","0.002","0.361","542","9186",

"AL","AMEA Sylacauga Plant","56018","3","2016","1498","0.005","0.677","1024.2","17365.5",

"AL","AMEA Sylacauga Plant","56018","4","2016","1405","0.005","0.586","884.1","14987.3",



In [111]:
epa = pd.read_csv(epa_path, index_col=False)

In [112]:
epa.head()

,State,Facility Name,Facility ID (ORISPL),Month,Year,Gross Load (MW-h),SO2 (tons),NOx (tons),CO2 (short tons),Heat Input (MMBtu)
0,AL,AMEA Sylacauga Plant,56018,1,2016,4534.0,0.014,2.229,3101.8,52585.8
1,AL,AMEA Sylacauga Plant,56018,2,2016,792.0,0.002,0.361,542.0,9186.0
2,AL,AMEA Sylacauga Plant,56018,3,2016,1498.0,0.005,0.677,1024.2,17365.5
3,AL,AMEA Sylacauga Plant,56018,4,2016,1405.0,0.005,0.586,884.1,14987.3
4,AL,AMEA Sylacauga Plant,56018,5,2016,1791.0,0.006,0.756,1145.1,19412.8


In [113]:
epa.tail()

,State,Facility Name,Facility ID (ORISPL),Month,Year,Gross Load (MW-h),SO2 (tons),NOx (tons),CO2 (short tons),Heat Input (MMBtu)
14476,WY,Wyodak,6101,8,2016,279246.0,233.875,347.809,321484.0,3065249.8
14477,WY,Wyodak,6101,9,2016,267691.0,228.710,334.644,309080.6,2946991.1
14478,WY,Wyodak,6101,10,2016,253110.0,211.495,314.882,290879.0,2773451.5
14479,WY,Wyodak,6101,11,2016,249476.0,207.293,306.374,283543.1,2703507.4
14480,WY,Wyodak,6101,12,2016,252072.0,207.199,304.010,282007.9,2688870.4


### Access parts of the dataframe

Look at the column names

In [114]:
epa.columns

Index(['State', ' Facility Name', ' Facility ID (ORISPL)', ' Month', ' Year',
       ' Gross Load (MW-h)', ' SO2 (tons)', ' NOx (tons)', ' CO2 (short tons)',
       ' Heat Input (MMBtu)'],
      dtype='object')

Notice that most of the columns have a leading space? We need to strip out those leading spaces and it might be nice to do some extra formatting.

In [115]:
epa.columns.str.strip()

Index(['State', 'Facility Name', 'Facility ID (ORISPL)', 'Month', 'Year',
       'Gross Load (MW-h)', 'SO2 (tons)', 'NOx (tons)', 'CO2 (short tons)',
       'Heat Input (MMBtu)'],
      dtype='object')

In [116]:
epa.columns = epa.columns.str.strip()

In [117]:
epa.columns = (epa.columns.str.lower()
                  .str.replace(' ', '_')
                  .str.replace('.', '')
                  .str.replace('-', '')
                  .str.replace('(', '')
                  .str.replace(')', ''))

### Data types of each column
Numeric columns will either be `int` or `float`. If a column is of type `object` it is either all strings or a mix of types. Watch out for columns that should be numeric but should up as `object`.

In [118]:
epa.dtypes

state                  object
facility_name          object
facility_id_orispl      int64
month                   int64
year                    int64
gross_load_mwh        float64
so2_tons              float64
nox_tons              float64
co2_short_tons        float64
heat_input_mmbtu      float64
dtype: object

## Basic statistics of the data

In [119]:
epa.describe()

,facility_id_orispl,month,year,gross_load_mwh,so2_tons,nox_tons,co2_short_tons,heat_input_mmbtu
count,14481.000000,14481.000000,14481.0,1.226200e+04,12465.000000,12700.000000,1.202200e+04,1.272200e+04
mean,22422.507769,6.490988,2016.0,1.974383e+05,117.769365,91.650835,1.582862e+05,1.743909e+06
std,24871.051651,3.442153,0.0,2.926614e+05,369.451131,214.962978,2.678391e+05,2.650419e+06
min,3.000000,1.000000,2016.0,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,2399.000000,4.000000,2016.0,4.474020e+03,0.028000,1.553750,4.555913e+03,5.153396e+04
50%,7145.000000,6.000000,2016.0,5.686740e+04,0.356000,9.534000,4.536591e+04,5.493512e+05
75%,55238.000000,9.000000,2016.0,2.890370e+05,16.868000,55.851500,1.799689e+05,2.454411e+06
max,70454.000000,12.000000,2016.0,2.093063e+06,5165.046000,2394.967000,2.341848e+06,2.232883e+07


Index into a dataframe using `.loc` or `.iloc` with square brackets and row,column notation

In [120]:
epa.iloc[0:5, :4]

,state,facility_name,facility_id_orispl,month
0,AL,AMEA Sylacauga Plant,56018,1
1,AL,AMEA Sylacauga Plant,56018,2
2,AL,AMEA Sylacauga Plant,56018,3
3,AL,AMEA Sylacauga Plant,56018,4
4,AL,AMEA Sylacauga Plant,56018,5


## Load capacity data

In [6]:
capacity = pd.read_excel(cap_path, sheet_name='Operable')

Looks like the first row isn't the column names and the last row is a footnote

In [7]:
capacity.head()

,"2016 Form EIA-860 Data - Schedule 3, 'Generator Data' (Operable Units Only)",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72
0,Utility ID,Utility Name,Plant Code,Plant Name,State,County,Generator ID,Technology,Prime Mover,Unit Code,...,Planned Energy Source 1,Planned New Nameplate Capacity (MW),Planned Repower Month,Planned Repower Year,Other Planned Modifications?,Other Modifications Month,Other Modifications Year,Multiple Fuels?,Cofire Fuels?,Switch Between Oil and Natural Gas?
1,195,Alabama Power Co,2,Bankhead Dam,AL,Tuscaloosa,1,Conventional Hydroelectric,HY,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
2,195,Alabama Power Co,3,Barry,AL,Mobile,1,Natural Gas Steam Turbine,ST,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
3,195,Alabama Power Co,3,Barry,AL,Mobile,2,Natural Gas Steam Turbine,ST,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
4,195,Alabama Power Co,3,Barry,AL,Mobile,4,Conventional Steam Coal,ST,NaN,...,NaN,,,,NaN,,,N,NaN,NaN


In [8]:
capacity.tail()

,"2016 Form EIA-860 Data - Schedule 3, 'Generator Data' (Operable Units Only)",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72
20721,61053,Mount Sinai Hospital,61416,Mount Sinai Hospital,NY,New York,GP8,Petroleum Liquids,IC,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
20722,61053,Mount Sinai Hospital,61416,Mount Sinai Hospital,NY,New York,GP9,Petroleum Liquids,IC,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
20723,61053,Mount Sinai Hospital,61416,Mount Sinai Hospital,NY,New York,HES13,Petroleum Liquids,IC,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
20724,61053,Mount Sinai Hospital,61416,Mount Sinai Hospital,NY,New York,HES14,Petroleum Liquids,IC,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
20725,NOTE: Information on planned capacity changes ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
capacity = pd.read_excel(cap_path, sheet_name='Operable', header=1, skipfooter=1)

In [38]:
capacity.head()

,Utility ID,Utility Name,Plant Code,Plant Name,State,County,Generator ID,Technology,Prime Mover,Unit Code,Ownership,Duct Burners,Can Bypass Heat Recovery Steam Generator?,RTO/ISO LMP Node Designation,RTO/ISO Location Designation for Reporting Wholesale Sales Data to FERC,Nameplate Capacity (MW),Nameplate Power Factor,Summer Capacity (MW),Winter Capacity (MW),Minimum Load (MW),Uprate or Derate Completed During Year,Month Uprate or Derate Completed,Year Uprate or Derate Completed,Status,Synchronized to Transmission Grid,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,Associated with Combined Heat and Power System,Sector Name,Sector,Topping or Bottoming,Energy Source 1,Energy Source 2,Energy Source 3,Energy Source 4,Energy Source 5,Energy Source 6,Startup Source 1,Startup Source 2,Startup Source 3,Startup Source 4,Solid Fuel Gasification System?,Carbon Capture Technology?,Turbines or Hydrokinetic Buoys,Time from Cold Shutdown to Full Load,Fluidized Bed Technology?,Pulverized Coal Technology?,Stoker Technology?,Other Combustion Technology?,Subcritical Technology?,Supercritical Technology?,Ultrasupercritical Technology?,Planned Net Summer Capacity Uprate (MW),Planned Net Winter Capacity Uprate (MW),Planned Uprate Month,Planned Uprate Year,Planned Net Summer Capacity Derate (MW),Planned Net Winter Capacity Derate (MW),Planned Derate Month,Planned Derate Year,Planned New Prime Mover,Planned Energy Source 1,Planned New Nameplate Capacity (MW),Planned Repower Month,Planned Repower Year,Other Planned Modifications?,Other Modifications Month,Other Modifications Year,Multiple Fuels?,Cofire Fuels?,Switch Between Oil and Natural Gas?
0,195,Alabama Power Co,2,Bankhead Dam,AL,Tuscaloosa,1,Conventional Hydroelectric,HY,NaN,S,X,X,NaN,NaN,53.9,0.92,56,52,52,N,,,OP,X,7,1963,,,N,Electric Utility,1,X,WAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,NaN,NaN,,,,NaN,,,N,NaN,NaN
1,195,Alabama Power Co,3,Barry,AL,Mobile,1,Natural Gas Steam Turbine,ST,NaN,S,X,X,NaN,NaN,153.1,0.85,55,55,55,N,,,OP,X,2,1954,12,2019,N,Electric Utility,1,X,NG,NaN,NaN,NaN,NaN,NaN,NG,NaN,NaN,NaN,NaN,NaN,,OVER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,NaN,NaN,,,,NaN,,,N,NaN,NaN
2,195,Alabama Power Co,3,Barry,AL,Mobile,2,Natural Gas Steam Turbine,ST,NaN,S,X,X,NaN,NaN,153.1,0.85,55,55,55,N,,,OP,X,7,1954,12,2019,N,Electric Utility,1,X,NG,NaN,NaN,NaN,NaN,NaN,NG,NaN,NaN,NaN,NaN,NaN,,OVER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,NaN,NaN,,,,NaN,,,N,NaN,NaN
3,195,Alabama Power Co,3,Barry,AL,Mobile,4,Conventional Steam Coal,ST,NaN,S,X,X,NaN,NaN,403.7,0.85,362,362,200,N,,,OP,X,12,1969,,,N,Electric Utility,1,X,BIT,NaN,NaN,NaN,NaN,NaN,NG,NaN,NaN,NaN,NaN,NaN,,OVER,NaN,Y,NaN,NaN,Y,NaN,NaN,,,,,,,,,NaN,NaN,,,,NaN,,,N,NaN,NaN
4,195,Alabama Power Co,3,Barry,AL,Mobile,5,Conventional Steam Coal,ST,NaN,S,X,X,NaN,NaN,788.8,0.85,738.5,738.5,465,N,,,OP,X,10,1971,,,N,Electric Utility,1,X,BIT,NaN,NaN,NaN,NaN,NaN,NG,NaN,NaN,NaN,NaN,NaN,,OVER,NaN,Y,NaN,NaN,NaN,Y,NaN,,,,,,,,,NaN,NaN,,,,NaN,,,N,NaN,NaN


### Check the column names and data types


In [14]:
capacity.columns

Index(['Utility ID', 'Utility Name', 'Plant Code', 'Plant Name', 'State',
       'County', 'Generator ID', 'Technology', 'Prime Mover', 'Unit Code',
       'Ownership', 'Duct Burners',
       'Can Bypass Heat Recovery Steam Generator?',
       'RTO/ISO LMP Node Designation',
       'RTO/ISO Location Designation for Reporting Wholesale Sales Data to FERC',
       'Nameplate Capacity (MW)', 'Nameplate Power Factor',
       'Summer Capacity (MW)', 'Winter Capacity (MW)', 'Minimum Load (MW)',
       'Uprate or Derate Completed During Year',
       'Month Uprate or Derate Completed', 'Year Uprate or Derate Completed',
       'Status', 'Synchronized to Transmission Grid', 'Operating Month',
       'Operating Year', 'Planned Retirement Month', 'Planned Retirement Year',
       'Associated with Combined Heat and Power System', 'Sector Name',
       'Sector', 'Topping or Bottoming', 'Energy Source 1', 'Energy Source 2',
       'Energy Source 3', 'Energy Source 4', 'Energy Source 5',
       'Ene

In [39]:
capacity.dtypes

Utility ID                                                                   int64
Utility Name                                                                object
Plant Code                                                                   int64
Plant Name                                                                  object
State                                                                       object
County                                                                      object
Generator ID                                                                object
Technology                                                                  object
Prime Mover                                                                 object
Unit Code                                                                   object
Ownership                                                                   object
Duct Burners                                                                object
Can 

In [20]:
type(capacity.dtypes)

pandas.core.series.Series

### It looks like several columns we would expect to be numeric are `object`

Pandas will only list the type of a column as `int` or `float` if all items can be cast that way. `object` means that either all items are strings (or another non-numeric, non-categorical type) or that the values are a mix of types.

Having numeric columns as `object` is a problem for us because operations like `groupby` won't work on non-numeric columns.

In [45]:
capacity.groupby('Plant Code').sum().head()

,Utility ID,Nameplate Capacity (MW),Operating Month,Operating Year,Sector
Plant Code,,,,,
2,195,53.9,7,1963,1
3,1950,2569.5,61,19848,10
4,585,225.0,23,5901,3
7,390,138.0,11,3898,2
8,585,1166.7,21,5886,3


### Finding non-numeric values with code
A search like this can also be done in the Excel or csv file. But sometimes it's easier to do with code.

Let's start by looking at the `Nameplate Power Factor` column and finding non-numeric entries. I'm using [this stackoverflow post](https://stackoverflow.com/questions/21771133/finding-non-numeric-rows-in-dataframe-in-pandas) as a template for my code below.

First use the `map` method to apply a function to every row of the Series. `map` (for Series), `apply` (for rows or columns of Dataframes), and `applymap` (for every element in a Dataframe) are powerful tools.

We'll use a `lambda` function here as part of the `map` method.

In [61]:
capacity['Nameplate Power Factor'].map(lambda x: isinstance(x, (int, float)))

0         True
1         True
2         True
3         True
4         True
5         True
6         True
7         True
8         True
9         True
10        True
11        True
12        True
13        True
14        True
15        True
16        True
17        True
18        True
19        True
20        True
21        True
22        True
23        True
24        True
25        True
26        True
27        True
28        True
29        True
30        True
31        True
32        True
33        True
34        True
35        True
36        True
37        True
38        True
39        True
40        True
41        True
42        True
43        True
44        True
45        True
46        True
47        True
48        True
49        True
         ...  
20674    False
20675    False
20676     True
20677    False
20678    False
20679    False
20680    False
20681    False
20682    False
20683     True
20684     True
20685     True
20686     True
20687     True
20688     True
20689     

Now we can use this boolean series, in conjunction with '~' for negation, to find all rows where values are not numeric.

In [62]:
capacity.loc[~capacity['Nameplate Power Factor'].apply(lambda x: isinstance(x, (int, float))), 'Nameplate Power Factor']

194       
195       
196       
286       
676       
912       
913       
1225      
1226      
1227      
1228      
1229      
1230      
1231      
1232      
1233      
1615      
1616      
1617      
1694      
1771      
2073      
2074      
2444      
2642      
3389      
3390      
3391      
3424      
3492      
3493      
3499      
3500      
3501      
3502      
3509      
3510      
3871      
3872      
3873      
4745      
4746      
4796      
4897      
6786      
7194      
7195      
7196      
7197      
7199      
        ..
20616     
20617     
20618     
20619     
20620     
20621     
20622     
20623     
20624     
20625     
20626     
20627     
20628     
20629     
20630     
20631     
20632     
20633     
20634     
20637     
20638     
20639     
20642     
20643     
20648     
20649     
20651     
20652     
20660     
20661     
20662     
20663     
20664     
20667     
20669     
20670     
20671     
20672     
20673     
20674     

This isn't very helpful though, so lets just look at the unique values.

In [63]:
capacity.loc[~capacity['Nameplate Power Factor'].apply(lambda x: isinstance(x, (int, float))), 'Nameplate Power Factor'].unique()

array([' '], dtype=object)

### Try loading capacity data again

In [65]:
capacity = pd.read_excel(cap_path, sheet_name='Operable', header=1, skipfooter=1,
                         na_values=' ')

In [68]:
capacity.dtypes

Utility ID                                                                   int64
Utility Name                                                                object
Plant Code                                                                   int64
Plant Name                                                                  object
State                                                                       object
County                                                                      object
Generator ID                                                                object
Technology                                                                  object
Prime Mover                                                                 object
Unit Code                                                                   object
Ownership                                                                   object
Duct Burners                                                                object
Can 

### Group capacity by plant code
EIA reports generation by fuel type and prime mover type, not necessarily by the actual prime mover. The EPA emissions data file I've included in this repository also groups data to the facility level. So lets group data to the facility level here to make everything easier.

In [135]:
capacity.groupby('Plant Code', as_index=False).sum().head()

,Plant Code,Utility ID,Nameplate Capacity (MW),Nameplate Power Factor,Summer Capacity (MW),Winter Capacity (MW),Minimum Load (MW),Month Uprate or Derate Completed,Year Uprate or Derate Completed,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,Sector,Turbines or Hydrokinetic Buoys,Planned Net Summer Capacity Uprate (MW),Planned Net Winter Capacity Uprate (MW),Planned Uprate Month,Planned Uprate Year,Planned Net Summer Capacity Derate (MW),Planned Net Winter Capacity Derate (MW),Planned Derate Month,Planned Derate Year,Planned New Nameplate Capacity (MW),Planned Repower Month,Planned Repower Year,Other Modifications Month,Other Modifications Year
0,2,195,53.9,0.920,56.0,52.0,52.0,0.0,0.0,7,1963,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,1950,2569.5,8.500,2172.5,2274.5,1421.0,0.0,0.0,61,19848,24.0,4038.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,585,225.0,2.571,225.9,228.0,225.0,0.0,0.0,23,5901,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7,390,138.0,1.700,130.0,130.0,56.0,0.0,0.0,11,3898,24.0,4038.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,585,1166.7,2.550,1053.7,1053.7,780.0,0.0,0.0,21,5886,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### What technology makes up the largest fraction of capacity at each plant?

There is probably a faster/more clever way to do this, but I'm going to:
- Loop through every plant/dataframe in a grouped object
- Identify the index with the largest summer capacity value
- If no summer capacity is given `idxmax` will return an error - use the nameplate capacity instead
- Build a new dataframe with this data

In [153]:
max_tech = {'plant_id': [],
            'technology': []}

for plant, _df in capacity.groupby('Plant Code'):
    max_tech['plant_id'].append(plant)
    
    try:
        tech = _df.loc[_df['Summer Capacity (MW)'].idxmax(), 'Technology']
    except TypeError:
        tech = _df.loc[_df['Nameplate Capacity (MW)'].idxmax(), 'Technology']
    max_tech['technology'].append(tech)

In [151]:
pd.DataFrame(max_tech)

,plant_id,technology
0,2,Conventional Hydroelectric
1,3,Conventional Steam Coal
2,4,Conventional Hydroelectric
3,7,Natural Gas Steam Turbine
4,8,Conventional Steam Coal
5,9,Natural Gas Fired Combustion Turbine
6,10,Natural Gas Steam Turbine
7,11,Conventional Hydroelectric
8,12,Conventional Hydroelectric
9,13,Conventional Hydroelectric


# Load generation data

In [72]:
generation = pd.read_excel(gen_path, header=5)

In [73]:
generation.head()

,Plant Id,Combined Heat And Power Plant,Nuclear Unit Id,Plant Name,Operator Name,Operator Id,Plant State,Census Region,NERC Region,Reserved,NAICS Code,EIA Sector Number,Sector Name,Reported Prime Mover,Reported Fuel Type Code,AER Fuel Type Code,Reserved.1,Reserved.2,Physical Unit Label,Quantity January,Quantity February,Quantity March,Quantity April,Quantity May,Quantity June,Quantity July,Quantity August,Quantity September,Quantity October,Quantity November,Quantity December,Elec_Quantity January,Elec_Quantity February,Elec_Quantity March,Elec_Quantity April,Elec_Quantity May,Elec_Quantity June,Elec_Quantity July,Elec_Quantity August,Elec_Quantity September,Elec_Quantity October,Elec_Quantity November,Elec_Quantity December,MMBtuPer_Unit January,MMBtuPer_Unit February,MMBtuPer_Unit March,MMBtuPer_Unit April,MMBtuPer_Unit May,MMBtuPer_Unit June,MMBtuPer_Unit July,MMBtuPer_Unit August,MMBtuPer_Unit September,MMBtuPer_Unit October,MMBtuPer_Unit November,MMBtuPer_Unit December,Tot_MMBtu January,Tot_MMBtu February,Tot_MMBtu March,Tot_MMBtu April,Tot_MMBtu May,Tot_MMBtu June,Tot_MMBtu July,Tot_MMBtu August,Tot_MMBtu September,Tot_MMBtu October,Tot_MMBtu November,Tot_MMBtu December,Elec_MMBtu January,Elec_MMBtu February,Elec_MMBtu March,Elec_MMBtu April,Elec_MMBtu May,Elec_MMBtu June,Elec_MMBtu July,Elec_MMBtu August,Elec_MMBtu September,Elec_MMBtu October,Elec_MMBtu November,Elec_MMBtu December,Netgen January,Netgen February,Netgen March,Netgen April,Netgen May,Netgen June,Netgen July,Netgen August,Netgen September,Netgen October,Netgen November,Netgen December,Total Fuel Consumption Quantity,Electric Fuel Consumption Quantity,Total Fuel Consumption MMBtu,Elec Fuel Consumption MMBtu,Net Generation (Megawatthours),YEAR
0,2,N,NaN,Bankhead Dam,Alabama Power Co,195,AL,ESC,SERC,NaN,22,1,Electric Utility,HY,WAT,HYC,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-77.112,-69.679,-48.374,-24.341,-11.476,-11.441,-11.593,-17.407,-11.566,-10.832,-10.27,-20.909,0,0,0,0,-325.000,2016
1,3,N,NaN,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,22,1,Electric Utility,CA,NG,NG,NaN,NaN,mcf,57253,27242,86461,54707,64489,90182,120407,115553,109091,67504,64673,20895,57253,27242,86461,54707,64489,90182,120407,115553,109091,67504,64673,20895,1.017,1.019,1.02,1.018,1.017,1.017,1.014,1.022,1.024,1.021,1.019,1.024,58226,27760,88190,55692,65585,91715,122093,118095,111709,68922,65902,21396,58226,27760,88190,55692,65585,91715,122093,118095,111709,68922,65902,21396,268797,251423,214816,261564,223662,206449,259415,229111,251839,219263,266927,265011,878457,878457,895285,895285,2918277.000,2016
2,3,N,NaN,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,22,1,Electric Utility,CT,NG,NG,NaN,NaN,mcf,5248798,4781915,4076291,5053433,4355654,4066516,5055348,4780696,4824758,4159500,5142640,5148943,5248798,4781915,4076291,5053433,4355654,4066516,5055348,4780696,4824758,4159500,5142640,5148943,1.017,1.019,1.02,1.018,1.017,1.017,1.014,1.022,1.024,1.021,1.019,1.024,5338028,4872771,4157817,5144395,4429700,4135647,5126123,4885871,4940552,4246850,5240350,5272518,5338028,4872771,4157817,5144395,4429700,4135647,5126123,4885871,4940552,4246850,5240350,5272518,511773,457142,396211,490439,421779,388783,477329,477474,463646,401506,495929,501072,56694492,56694492,57790622,57790622,5483083.000,2016
3,3,N,NaN,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,22,1,Electric Utility,ST,BIT,COL,NaN,NaN,short tons,159951,129081,137723,19982,178956,241627,260989,273207,228027,144161,169610,100688,159951,129081,137723,19982,178956,241627,260989,273207,228027,144161,169610,100688,20.589,20.736,20.851,20.838,21.425,21.299,21.405,21.317,20.995,20.742,21.042,21.278,3293231,2676624,2871662,416385,3834132,5146413,5586470,5823954,4787427,2990187,3568934,2142439,3293231,2676624,2871662,416385,3834132,5146413,5586470,5823954,4787427,2990187,3568934,2142439,329513,259648,291745,35269.8,376727,510204,559747,583589,473794,286736,363877,207463,2

In [74]:
generation.columns

Index(['Plant Id', 'Combined Heat And\nPower Plant', 'Nuclear Unit Id',
       'Plant Name', 'Operator Name', 'Operator Id', 'Plant State',
       'Census Region', 'NERC Region', 'Reserved', 'NAICS Code',
       'EIA Sector Number', 'Sector Name', 'Reported\nPrime Mover',
       'Reported\nFuel Type Code', 'AER\nFuel Type Code', 'Reserved.1',
       'Reserved.2', 'Physical\nUnit Label', 'Quantity\nJanuary',
       'Quantity\nFebruary', 'Quantity\nMarch', 'Quantity\nApril',
       'Quantity\nMay', 'Quantity\nJune', 'Quantity\nJuly', 'Quantity\nAugust',
       'Quantity\nSeptember', 'Quantity\nOctober', 'Quantity\nNovember',
       'Quantity\nDecember', 'Elec_Quantity\nJanuary',
       'Elec_Quantity\nFebruary', 'Elec_Quantity\nMarch',
       'Elec_Quantity\nApril', 'Elec_Quantity\nMay', 'Elec_Quantity\nJune',
       'Elec_Quantity\nJuly', 'Elec_Quantity\nAugust',
       'Elec_Quantity\nSeptember', 'Elec_Quantity\nOctober',
       'Elec_Quantity\nNovember', 'Elec_Quantity\nDecember',
   

In [75]:
generation.dtypes

Plant Id                                 int64
Combined Heat And\nPower Plant          object
Nuclear Unit Id                        float64
Plant Name                              object
Operator Name                           object
Operator Id                              int64
Plant State                             object
Census Region                           object
NERC Region                             object
Reserved                               float64
NAICS Code                               int64
EIA Sector Number                        int64
Sector Name                             object
Reported\nPrime Mover                   object
Reported\nFuel Type Code                object
AER\nFuel Type Code                     object
Reserved.1                             float64
Reserved.2                             float64
Physical\nUnit Label                    object
Quantity\nJanuary                       object
Quantity\nFebruary                      object
Quantity\nMar

In [76]:
generation.loc[~generation['Netgen\nJanuary'].map(lambda x: isinstance(x, (int, float))), 'Netgen\nJanuary'].unique()

array(['.'], dtype=object)

### Melt generation data to tidy format and groupby facility

In [77]:
generation = pd.read_excel(gen_path, header=5, na_values='.')

In [78]:
generation.columns = ((generation.columns.str.strip()
                                 .str.lower()
                                 .str.replace('\n', ' ')
                                 .str.replace(' ', '_')
                                 .str.replace('-', '')
                                 .str.replace('(', '')
                                 .str.replace(')', '')))

In [79]:
value_cols = [col for col in generation.columns if 'netgen' in col]

In [80]:
df = (pd.melt(generation, id_vars=['plant_id'],
              value_vars=value_cols, value_name='net_gen',
              var_name='month')
        .groupby(['plant_id', 'month'], as_index=False)
        .sum())

In [178]:
pd.melt(generation, id_vars=['plant_id', 'aer_fuel_type_code'],
              value_vars=value_cols, value_name='net_gen',
              var_name='month')

,plant_id,aer_fuel_type_code,month,net_gen
0,2,HYC,netgen_january,-77.112
1,3,NG,netgen_january,268797.000
2,3,NG,netgen_january,511773.000
3,3,COL,netgen_january,329513.400
4,3,NG,netgen_january,13195.605
5,4,HYC,netgen_january,139641.000
6,7,COL,netgen_january,0.000
7,7,NG,netgen_january,14290.000
8,7,WWW,netgen_january,0.000
9,8,COL,netgen_january,460697.090


In [81]:
df['month'] = df.month.str.replace('netgen_', '')

In [82]:
df.head()

,plant_id,month,net_gen
0,2,april,-24.341
1,2,august,-17.407
2,2,december,-20.909
3,2,february,-69.679
4,2,january,-77.112


### Convert month columns to integer values
The EPA emissions data also has a `month` column but the values are integers. We'll use a built-in list of months from the `calendar` package to create a maping of names to integers.

In [121]:
epa.month.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [97]:
import calendar

In [102]:
calendar.month_name

In [99]:
list(calendar.month_name)

['',
 'January',
 'February',
 'March',
 'April',
 'May',
 'June',
 'July',
 'August',
 'September',
 'October',
 'November',
 'December']

In [103]:
# Will need to capitalize the month names to match the list
df.month.str.capitalize().head()

0       April
1      August
2    December
3    February
4     January
Name: month, dtype: object

In [124]:
month_map = {month: idx for idx, month in enumerate(calendar.month_name)}
month_map

{'': 0,
 'January': 1,
 'February': 2,
 'March': 3,
 'April': 4,
 'May': 5,
 'June': 6,
 'July': 7,
 'August': 8,
 'September': 9,
 'October': 10,
 'November': 11,
 'December': 12}

In [126]:
df.month.str.capitalize().head().map(month_map)

0     4
1     8
2    12
3     2
4     1
Name: month, dtype: int64

In [127]:
df['month'] = df.month.str.capitalize().map(month_map)

In [128]:
df.head()

,plant_id,month,net_gen
0,2,4,-24.341
1,2,8,-17.407
2,2,12,-20.909
3,2,2,-69.679
4,2,1,-77.112


In [130]:
df.sort_values(['plant_id', 'month'], inplace=True)

In [131]:
df.month.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [132]:
df.head()

,plant_id,month,net_gen
4,2,1,-77.112
3,2,2,-69.679
7,2,3,-48.374
0,2,4,-24.341
8,2,5,-11.476


### What fuel is used to generate the most electricity at each plant?

In [188]:
df_fuel = (pd.melt(generation, id_vars=['plant_id', 'aer_fuel_type_code'],
              value_vars=value_cols, value_name='net_gen',
              var_name='month')
             .groupby(['plant_id', 'aer_fuel_type_code'], as_index=False)
             .sum())

In [181]:
_df_fuel = pd.melt(generation, id_vars=['plant_id', 'aer_fuel_type_code'],
              value_vars=value_cols, value_name='net_gen',
              var_name='month')

In [190]:
df_fuel.head()

,plant_id,aer_fuel_type_code,net_gen
0,2,HYC,-325.000
1,3,COL,4278313.069
2,3,NG,8492577.936
3,4,HYC,468960.000
4,7,COL,0.000


In [182]:
_df_fuel.head()

,plant_id,aer_fuel_type_code,month,net_gen
0,2,HYC,netgen_january,-77.112
1,3,NG,netgen_january,268797.000
2,3,NG,netgen_january,511773.000
3,3,COL,netgen_january,329513.400
4,3,NG,netgen_january,13195.605


In [172]:
max_fuel_index = df_fuel.groupby('plant_id')['net_gen'].idxmax()
max_fuel_index.head()

plant_id
2    0
3    2
4    3
7    5
8    7
Name: net_gen, dtype: int64

In [175]:
max_fuel = df_fuel.loc[max_fuel_index, ['aer_fuel_type_code', 'plant_id']]

In [176]:
df.merge(max_fuel, on='plant_id')

,plant_id,month,net_gen,aer_fuel_type_code
0,2,1,-77.112,HYC
1,2,2,-69.679,HYC
2,2,3,-48.374,HYC
3,2,4,-24.341,HYC
4,2,5,-11.476,HYC
5,2,6,-11.441,HYC
6,2,7,-11.593,HYC
7,2,8,-17.407,HYC
8,2,9,-11.566,HYC
9,2,10,-10.832,HYC
